In [32]:
import numpy as np
from scipy import spatial
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [60]:
window_items =  [256, 1, 257, 6, 10, 12, 205, 206, 207, 210, 84, 212, 213, 214, 216, 217, 220, 121, 233, 234, 235, 239, 241, 243, 244, 245, 246, 247, 248, 249, 251, 252, 253, 254, 255] 
target_items = [218, 286, 299, 307, 318, 658, 675, 805]

In [61]:
import os,sys
my_path = os.getcwd()
print (my_path)
path = os.path.join(my_path, "../data/ratings.txt")
print (path)

/media/thepurpleowl/FCC8B519C8B4D35E/Academics_research/research/BKPatra/Attacks_RS/AttackProfileDetectionRS/filmtrust/Item-Based-Recommendation
/media/thepurpleowl/FCC8B519C8B4D35E/Academics_research/research/BKPatra/Attacks_RS/AttackProfileDetectionRS/filmtrust/Item-Based-Recommendation/../data/ratings.txt


In [87]:
train = pd.read_csv(path,sep=' ',header=None,names=['userId','itemId','rating'])

In [88]:
train.describe()

,userId,itemId,rating
count,35497.000000,35497.000000,35497.000000
mean,744.247824,265.524946,3.002803
std,433.199457,346.854769,0.918692
min,1.000000,1.000000,0.500000
25%,356.000000,12.000000,2.500000
50%,735.000000,216.000000,3.000000
75%,1130.000000,251.000000,4.000000
max,1508.000000,2071.000000,4.000000


### Actual Average Rating of each item 

In [89]:
calculated_Rating = {}
items = train['itemId'].unique()

#rating
R = train.groupby(['itemId'])['rating'].mean()
for i in items:
#     print (i)
    calculated_Rating[i] = R[i]
    
# print (calculated_Rating)

In [90]:
test = pd.DataFrame()
for t_i in target_items:
    temp_df = train[train['itemId'] == t_i]
    train =  train.drop(train[train['itemId'] == t_i].index)
    test = pd.concat([temp_df, test]).drop_duplicates(keep=False)

### testing code of divising test/target data
table = [['Heading1', 'Heading2'], [1 , 1],[1 , 2],[1 , 3],[1 , 4], [1,5],[2 , 1],[2 , 2],[2 , 3],[2 , 4],[3, 1],[3, 2],[3, 3],[3, 4]]  
headers = table.pop(0)  
df = pd.DataFrame(table, columns=headers)

In [91]:
dfs = {n: g for n, g in test.groupby(test.groupby('itemId').cumcount() % 10)} 
test = pd.concat([dfs[0],dfs[1]], ignore_index=False)
train = pd.concat([train,dfs[2],dfs[3],dfs[4],dfs[5]], ignore_index=False)
del_data = pd.concat([dfs[6],dfs[7],dfs[8],dfs[9]], ignore_index=False)

In [67]:
train.shape[0]

35321

In [92]:
test.shape[0]

68

In [93]:
test.tail(5)

,userId,itemId,rating
14461,591,218,4.0
19110,795,218,4.0
24891,1041,218,4.0
28918,1210,218,4.0
34036,1456,218,4.0


In [94]:
train.tail(5)

,userId,itemId,rating
11557,472,218,1.5
16805,687,218,4.0
20372,845,218,4.0
25733,1084,218,4.0
31684,1341,218,3.5


In [95]:
del_data.tail(5)

,userId,itemId,rating
14213,578,218,4.0
18554,769,218,3.0
23045,965,218,4.0
28047,1183,218,4.0
33446,1432,218,3.0


### Finding error for target items in actual data

In [96]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{675: [0, 0], 658: [0, 0], 307: [0, 0], 286: [0, 0], 218: [0, 0], 299: [0, 0], 318: [0, 0], 805: [0, 0]}


In [73]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['itemId']==usr]
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(list_of_movies_current_user_has_rated.itertuples(),1): 
        c1=[]
        c2=[]
        #c_user=int(u['userId'])
        c_item=int(u.itemId)
        #list_curuser=train[train['userId']==c_user]
        list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( all_ratings_for_the_active_item,list_curitem,how='inner',on=['userId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

In [74]:
index

68

In [75]:
print (t2-t1)

5.69273829460144


In [76]:
target_item_error_in_actual_data

{218: [43.254042504042502, 16],
 286: [22.565217391304348, 6],
 299: [15.238095238095237, 5],
 307: [16.698287220026351, 6],
 318: [35.599206349206348, 12],
 658: [15.530138339920949, 6],
 675: [20.022357723577237, 9],
 805: [23.939285714285717, 8]}

### MAE error

In [77]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

675     : 2.22470641373
658     : 2.58835638999
307     : 2.78304787
286     : 3.76086956522
218     : 2.7033776565
299     : 3.04761904762
318     : 2.9666005291
805     : 2.99241071429


## Nuke Attack Strength Check

### Injecting Attack Profiles

In [78]:
nuke_rating = 0.5

In [79]:
attack_ids = list(range(1600,1700))

for attacker in attack_ids:
    for item in window_items:
        a = calculated_Rating[item]
        attack_rating = int(a) if (a-int(a)<0.5) else int(a)+1
        train = train.append(pd.Series([attacker,item,attack_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

for attacker in attack_ids:
    for item in target_items:
        train = train.append(pd.Series([attacker,item,nuke_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

In [80]:
train.shape[0]

39621

In [99]:
train.head(5)

,userId,itemId,rating
0,1,1,2.0
1,1,2,4.0
2,1,3,3.5
3,1,4,3.0
4,1,5,4.0


## Finding error for target items in attack profile injected data

In [82]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{675: [0, 0], 658: [0, 0], 307: [0, 0], 286: [0, 0], 218: [0, 0], 299: [0, 0], 318: [0, 0], 805: [0, 0]}


In [83]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['itemId']==usr]
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(list_of_movies_current_user_has_rated.itertuples(),1): 
        c1=[]
        c2=[]
        #c_user=int(u['userId'])
        c_item=int(u.itemId)
        #list_curuser=train[train['userId']==c_user]
        list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( all_ratings_for_the_active_item,list_curitem,how='inner',on=['userId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

In [84]:
print (t2-t1)

5.4322898387908936


In [85]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

675     : 2.22470641373
658     : 2.58835638999
307     : 2.78304787
286     : 3.76086956522
218     : 2.7033776565
299     : 3.04761904762
318     : 2.9666005291
805     : 2.99241071429
